In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io
import oracledb

In [2]:

def create_oracle_connection():
    try:
        # Inicializar el cliente Oracle en modo "thick"
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        
        # Crear la conexión
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        
        print("Conexión exitosa.")
        return connection
    
    except oracledb.DatabaseError as e:
        # Captura y muestra cualquier error que ocurra durante la conexión
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None
    
# Uso de la función para crear una conexión
conn = create_oracle_connection()

# Asegúrate de cerrar la conexión cuando termines
if conn:
    conn.close()

Conexión exitosa.


In [3]:
# Definición de la función de conexión
def create_oracle_connection():
    try:
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        print("Conexión exitosa.")
        return connection
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None

# Reemplazar fetch_data por una función de ejemplo
def fetch_data(fec_ini, fec_fin, cas, limit=None):
    try:
        conn = create_oracle_connection()
        if conn is None:
            raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
        query = f"""
            SELECT A.ATENAMBORICENASICOD AS ORIGEN,
                   A.ATENAMBCENASICOD AS CENTRO,
                   TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
                   DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
                   CB.GRPETA1COD AS GETAREO,
                   Y.PERSECNUM AS PACSECNUM,
                   SR.SERVHOSCOD AS CODSER,
                   SR.SERVHOSTIPPROCOD AS TIPPROF 
            FROM CTAAM10 A  
            LEFT OUTER JOIN CMAME10 X
              ON X.ORICENASICOD = A.ATENAMBORICENASICOD
             AND X.CENASICOD = A.ATENAMBCENASICOD
             AND X.ACTMEDNUM = A.ATENAMBNUM
            LEFT OUTER JOIN CMPER10 Y
              ON X.ACTMEDPACSECNUM = Y.PERSECNUM
            LEFT OUTER JOIN SGSS.CBGPE10 CB
              ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
            LEFT OUTER JOIN CMSHO10 SR
              ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
            WHERE A.ATENAMBATENFEC BETWEEN TO_DATE('{fec_ini}', 'DD-MM-YYYY') AND
                  TO_DATE('{fec_fin}', 'DD-MM-YYYY')
              AND A.ATENAMBESTREGCOD = '1'
              AND Y.PERSECNUM IS NOT NULL
              AND A.ATENAMBCENASICOD ='{cas}'
        """
        if limit:
            query += f" AND ROWNUM <= {limit}"

        df = pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error: {e}")
        df = pd.DataFrame()
    finally:
        conn.close()  # Cerrar la conexión al final
    return df



In [4]:
# Crear la aplicación Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Consulta de Datos Médicos"),

    html.Div([
        html.Label("Fecha Inicial:"),
        dcc.DatePickerSingle(id='fec_ini', date='2024-07-01',display_format='DD/MM/YYYY'),
        html.Label("Fecha Final:"),
        dcc.DatePickerSingle(id='fec_fin', date='2024-07-02',display_format='DD/MM/YYYY'),
        html.Label("Centro:"),
        dcc.Input(id='cas', type='text', value='004'),
        html.Button('Buscar', id='submit-val', n_clicks=0),
        html.Button('Descargar CSV', id='download-btn', n_clicks=0),
    ]),

    html.Div(id='output-data-table'),
    dcc.Download(id="download-dataframe-csv"),
])

@app.callback(
    Output('output-data-table', 'children'),
    Input('submit-val', 'n_clicks'),
    State('fec_ini', 'date'),
    State('fec_fin', 'date'),
    State('cas', 'value')
)
def update_output(n_clicks, fec_ini, fec_fin, cas):
    if n_clicks > 0:
        try:
            if not fec_ini or not fec_fin or not cas:
                return html.Div("Por favor, complete todos los campos.")
            
            df = fetch_data(fec_ini, fec_fin, cas, limit=100)  # Limitar a las primeras 100 filas

            if df.empty:
                return html.Div("No se encontraron datos para los criterios proporcionados.")
            
            return html.Div([
                html.H4("Resultados:"),
                dash_table.DataTable(
                    id='data-table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    style_table={'overflowX': 'auto'},
                    page_size=10  # Número de filas por página
                )
            ])
        except Exception as e:
            return html.Div(f'Error: {e}')

@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-btn", "n_clicks"),
    State('fec_ini', 'date'),
    State('fec_fin', 'date'),
    State('cas', 'value'),
    prevent_initial_call=True
)
def download_csv(n_clicks, fec_ini, fec_fin, cas):
    if n_clicks > 0:
        try:
            if not fec_ini or not fec_fin or not cas:
                return dcc.send_data_frame(pd.DataFrame().to_csv, filename="data_empty.csv")

            df_complete = fetch_data(fec_ini, fec_fin, cas)  # Sin límite para la descarga
            return dcc.send_data_frame(df_complete.to_csv, filename="data_complete.csv")
        except Exception as e:
            return dcc.send_data_frame(pd.DataFrame().to_csv, filename="data_empty.csv")

if __name__ == '__main__':
    app.run_server(debug=True)